#SETUP

Downloading files to be sent:

In [1]:
!ls

Podington_Bear_-_Starling.mp3  sender_stats.txt   stop&wait.mp3
receiver_stats.txt	       Socket_Prog.ipynb  test_file.txt


In [2]:
!ls

Podington_Bear_-_Starling.mp3  sender_stats.txt   stop&wait.mp3
receiver_stats.txt	       Socket_Prog.ipynb  test_file.txt


In [3]:
#!wget https://www.dropbox.com/s/4fm193mtun5nzl0/big_text_file.txt  # big_text_file.txt file of 4.00 MB that will be sent
#!wget https://www.dropbox.com/s/g8oq6mm2j1zhkv2/test_file.txt      # small test text file
#!wget https://freemusicarchive.org/file/music/Music_for_Video/Podington_Bear/Solo_Instruments/Podington_Bear_-_Starling.mp3  # +4 MB mp3 file

sender and receiver:

In [4]:
import socket, pickle
receiverIP=socket.gethostbyname(socket.gethostname())
from socket import *
senderPort = 12000
receiverPort = 5200 

packets and Acks classes:

In [5]:
class Packet:
  def __init__(
  self,
  seq_num=None,
  checksum=None,
  data=None,
  type_ =1
  ):
    self.seq_num=seq_num
    self.checksum=checksum
    self.data=data
    self.type_=type_

class ACK:
  def __init__(
  self,
  seq_num=None,
  checksum=None,
  real=None,
  data=None
  ):
    self.seq_num=seq_num
    self.checksum=checksum
    self.real=real
    self.data=data
  

checksum calculation trial:

In [6]:
#!pip install hashlib
import struct
import hashlib
f = open("test_file.txt", 'rb')
#read one byte in bunary format
data = f.read(1)
print(data)

x=hashlib.sha224(data).hexdigest()
y=hashlib.sha224(data).hexdigest()


print(x == y)

b'k'
True


randomly dropping packets function:

In [7]:
import random

def decision(probability):
    return random.random() < probability

Dividing files to packets and vice versa:

and writing logs:

In [8]:
import csv

def file_to_pkt(file_name, sw, seq_num_range):
    packets = []
    f = open(file_name, 'rb')
    data = f.read(1024)
    seqNO = 0
    while data:
        if sw:
            packets.append(Packet(seqNO % 2, hashlib.sha224(data).hexdigest(), data))

        else: #added checksum instead of 0
            packets.append(Packet(seqNO % seq_num_range, hashlib.sha224(data).hexdigest(), data))
        data = f.read(1024)
        
        seqNO += 1
        
    packets[len(packets) - 1].type_ = 0
    return packets
  

def pkt_to_file(file_name, packets):
    f = open(file_name, 'wb')
    for i in range(len(packets)):
        f.write(packets[i].data)
    f.close()
    
def get_stats(file_name,stats_dict):
  with open(file_name, "w") as f:
    writer = csv.writer(f, delimiter="\t", lineterminator="\r\n")
    writer.writerows(stats_dict.items())
    


#STOP AND WAIT

In [9]:
#@title Parameters Tuning { form-width: "30%"}
sent_file_name = "Podington_Bear_-_Starling.mp3" #@param {type:"string"}
recv_file_name = "stop&wait.mp3" #@param {type:"string"}
prob = 0.9 #@param {type:"slider", min:0, max:1, step:0.01}
variable_name = 0


import os
import time


start_time = time.time()

  
def recv_child():
    stats_ = {}
    receiverSocket = socket(AF_INET, SOCK_DGRAM)
    receiverSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    receiverSocket.bind(('', receiverPort))
    
    print ('--The receiver is ready to receive--\n')

    packets = []
    actual_seq_num = 0
    expected_seq_num = 0
    while True:
        (message, senderAddress) = receiverSocket.recvfrom(4096)
        #if a packet is received
        message = pickle.loads(message)
        #if checksum is correct
        if(message.checksum == hashlib.sha224(message.data).hexdigest()):
          
          t6 = time.time()

          if expected_seq_num == message.seq_num:
              packets.append(message)

              if decision(prob):
                  receiverSocket.sendto(pickle.dumps(ACK(expected_seq_num,
                          message.checksum, actual_seq_num,message.data)), senderAddress)
                  t6 = time.time()
                  stats_['{0:.6f}'.format(t6 - start_time)] = \
                      'receiver sent ack for packet ' \
                      + str(expected_seq_num) + ' actual: ' \
                      + str(actual_seq_num)
                  actual_seq_num += 1
                  expected_seq_num = actual_seq_num % 2

                  if message.type_ == 0:
                      time.sleep(0.1)

                      print ('--Whole file received successfully. File is ready to download--\n')

                      get_stats('receiver_stats.txt', stats_)

                      pkt_to_file(recv_file_name, packets)
              else:

                  stats_['{0:.6f}'.format(t6 - start_time)] = \
                      'receiver dropped ack for packet ' \
                      + str(expected_seq_num) + ' actual: ' \
                      + str(actual_seq_num)
                  actual_seq_num += 1
                  expected_seq_num = actual_seq_num % 2
          else:

              # duplicate

              receiverSocket.sendto(pickle.dumps(ACK(message.seq_num, message.checksum,
                                    actual_seq_num - 1,message.data)), senderAddress)

              stats_['{0:.6f}'.format(t6 - start_time)] = \
                  '--duplicate-- already received ' \
                  + str(message.seq_num) + ' actual: ' \
                  + str(actual_seq_num - 1)

        else:
          print("wrong checksum detected for packet : ",message.seq_num)
          print("expected : ",message.checksum)
          print("received : ",hashlib.sha224(message.data).hexdigest() )
      

def send_parent():
  
    newpid = os.fork()
    if newpid == 0:
        recv_child()
    else:
        stats_ = {}
        time.sleep(3)
        pids = (os.getpid(), newpid)
        packets = []

        packets = file_to_pkt(file_name=sent_file_name, sw=True,  seq_num_range= 0)

        senderSocket = socket(AF_INET, SOCK_DGRAM)
        senderSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
        senderSocket.bind(('', senderPort))
        message = packets

        for i in range(len(packets)):
            if decision(prob):
                senderSocket.sendto(pickle.dumps(message[i]),
                                    (receiverIP, receiverPort))
                #print("sending message of data :",message[i].data)
                #print("sending message of checksum :",message[i].checksum)
                #print("check should be ",hashlib.sha224(message[i].data).hexdigest())
                t1 = time.time()
                stats_['{0:.6f}'.format(t1 - start_time)] = 'sent packet ' \
                    + str(i % 2) + ' actual: ' + str(i)
            else:
                t1 = time.time()
                stats_['{0:.6f}'.format(t1 - start_time)] = 'sent packet ' \
                    + str(i % 2) + ' actual: ' + str(i)

            while True:

                modifiedMessage = pot_ack = None
                try:
                    senderSocket.settimeout(0.01)
                    (modifiedMessage, receiverAddress) = \
                        senderSocket.recvfrom(4096)
                    #receive a packet
                    t5 = time.time()
                    pot_ack = pickle.loads(modifiedMessage)
                    #checking checksum
                    if(pot_ack.checksum == hashlib.sha224(pot_ack.data).hexdigest() ):
                      
                      if isinstance(pot_ack, ACK):
                          if pot_ack.seq_num == i % 2:
                              stats_['{0:.6f}'.format(t5 - start_time)] = \
                                  'sender received ack for packet ' \
                                  + str(pot_ack.seq_num) + ' actual: ' \
                                  + str(pot_ack.real)
                              modifiedMessage = None

                              if i == len(packets) - 1:
                                  get_stats('sender_stats.txt', stats_)
                                  print ('--Whole file sent successfully--\n')

                          break
                          
                      else:
                        print("wrong checksum detected for Ack : ",pot_ack.real,pot_ack.seq_num)
                        print("expected : ",pot_ack.checksum)
                        print("received : ",hashlib.sha224(pot_ack.data).hexdigest() )
                        
                except:
                    senderSocket.settimeout(None)
                    t8 = time.time()
                    stats_['{0:.6f}'.format(t8 - start_time)] = \
                        '--timeout-- no ack received for packet ' \
                        + str(i % 2) + ' actual: ' + str(i)

                    if decision(prob):
                        senderSocket.sendto(pickle.dumps(message[i]),
                                (receiverIP, receiverPort))
                        t7 = time.time()
                        stats_['{0:.6f}'.format(t7 - start_time)] = \
                            '--again-- sent packet ' + str(i % 2) \
                            + ' actual: ' + str(i)
                    else:
                        t7 = time.time()
                        stats_['{0:.6f}'.format(t7 - start_time)] = \
                            '--again-- dropped packet ' + str(i % 2) \
                            + ' actual: ' + str(i)

    senderSocket.close()


send_parent()


--The receiver is ready to receive--

--Whole file sent successfully--

--Whole file received successfully. File is ready to download--

--Whole file received successfully. File is ready to download--



In [ ]:
#!ps -fA | grep python
from google.colab import files
files.download(recv_file_name)
files.download('receiver_stats.txt')
files.download('sender_stats.txt')

#GBN

In [ ]:
from threading import Thread, Event, Timer, Lock
import time

def TimerReset(*args, **kwargs):
    """ Global function for Timer """
    return _TimerReset(*args, **kwargs)


class _TimerReset(Thread):

    def __init__(self, interval, function, args=[], kwargs={}):
        Thread.__init__(self)
        self.interval = interval
        self.function = function
        self.args = args
        self.kwargs = kwargs
        self.finished = Event()
        self.resetted = True

    def cancel(self):
       
        self.finished.set()

    def run(self):
        while not self.finished.isSet():
            
    
            self.resetted = True
            while self.resetted:
                
                self.resetted = False
                self.finished.wait(self.interval)
    
            if not self.finished.isSet():
                self.function(*self.args, **self.kwargs)
        

    def reset(self, interval=None):
        """ Reset the timer """

        if interval:
            
            self.interval = interval
        else:
            
            pass

        self.resetted = True
        self.finished.set()
        self.finished.clear()

In [ ]:
lock = Lock()

def resend(Packet,prob,start_time,stats_,senderSocket):

    if decision(prob):
      with lock:
        senderSocket.sendto(pickle.dumps(Packet),(receiverIP, receiverPort))
        t7 = time.time()
        stats_['{0:.6f}'.format(t7 - start_time)] = \
        'sent packet ' + str(Packet.seq_num) \
        + ' actual: ' + str(Packet.act_num)
        print(stats_['{0:.6f}'.format(t7 - start_time)])

    else:
      t7 = time.time()
      stats_['{0:.6f}'.format(t7 - start_time)] = \
      'dropped packet ' + str(Packet.seq_num) \
      + ' actual: ' + str(Packet.act_num)

    
    return 
        

In [ ]:
#@title Parameters Tuning { form-width: "40%"}
sent_file_name = "Podington_Bear_-_Starling.mp3" #@param {type:"string"}
recv_file_name = "gbn.mp3" #@param {type:"string"}
prob = 1 #@param {type:"slider", min:0, max:1, step:0.01}
seq_num_range = 8 #@param {type:"integer"}
window_size = 4 #@param {type:"integer"}
time_out = 0.05 #@param {type:"number"}

import heapq
import os
import time
from google.colab import files

start_time = time.time()


def recv_child():
    stats_ = {}
    receiverSocket = socket(AF_INET, SOCK_DGRAM)
    receiverSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    receiverSocket.bind(('', receiverPort))
    
    print ('--The receiver is ready to receive--\n')

    packets = []
    actual_seq_num = 0
    expected_seq_num = 0
    while True:

        (message, senderAddress) = receiverSocket.recvfrom(1024*(window_size+1))
        message = pickle.loads(message)
        t6 = time.time()
        
        if expected_seq_num == message.seq_num:
            packets.append(message)
            if decision(prob):
                receiverSocket.sendto(pickle.dumps(ACK(expected_seq_num,
                        0, actual_seq_num)), senderAddress)
                t6 = time.time()
                stats_['{0:.6f}'.format(t6 - start_time)] = \
                    'receiver sent ack for packet ' \
                    + str(expected_seq_num) + ' actual: ' \
                    + str(actual_seq_num)
                actual_seq_num += 1
                expected_seq_num = actual_seq_num % seq_num_range

                if message.type_ == 0:
                    time.sleep(0.1)
                    
                    print ('--Whole file received successfully. File is ready to download--\n')

                    get_stats('receiver_stats.txt', stats_)

                    pkt_to_file(recv_file_name, packets)
            else:

                stats_['{0:.6f}'.format(t6 - start_time)] = \
                    'receiver dropped ack for packet ' \
                    + str(expected_seq_num) + ' actual: ' \
                    + str(actual_seq_num)
                actual_seq_num += 1
                expected_seq_num = actual_seq_num % seq_num_range
        else:

            # duplicate

            receiverSocket.sendto(pickle.dumps(ACK(message.seq_num, 0,
                                  actual_seq_num - 1)), senderAddress)

            stats_['{0:.6f}'.format(t6 - start_time)] = \
                '--out_of_order-- received ' \
                + str(message.seq_num) + ' expected: ' \
                + str(expected_seq_num) 



#Sender
def send_parent():
    #sender process
    newpid = os.fork()
    #if in child
    if newpid == 0:
        recv_child()
    else:#parent
        stats_ = {}
        time.sleep(1)
        packets = file_to_pkt(file_name=sent_file_name, sw=False,
                              seq_num_range=seq_num_range)

        senderSocket = socket(AF_INET, SOCK_DGRAM)
        senderSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
        senderSocket.bind(('', senderPort))

        # ## initial setting of window

        base = 0
        index = 0
        step = 0
        window = []
        tm=None
        not_acked = []
        acked_num = 0

        if index + window_size < len(packets):
            window = packets[index:(index + window_size)]
        else:
            window = packets[index:len(packets)]

        for i in range(len(window)):
            #for each packet in window create an event object and  a timer/thread
            #key of these dics to get these object is packet sequence number
            senderSocket.sendto(pickle.dumps(window[i]),(receiverIP, receiverPort))
           

            not_acked.append(window[i].seq_num)
        
        tm= TimerReset(time_out,resend,[window,prob,start_time,stats_,senderSocket])
        tm.start()
        while True:

            reply = None
            with lock:
              (reply, receiverAddress) = senderSocket.recvfrom(4096)
          
            reply = pickle.loads(reply)
            print("ack seq_num:",reply.seq_num,reply.real,"REC. ACK")
            if isinstance(reply,ACK):
               
                

                if (reply.seq_num in not_acked):
                    
                    not_acked.remove(reply.seq_num)

                    acked_num += 1

                    if acked_num == len(packets):
                        tm.cancel()
                        senderSocket.close()
                        break

                    if base == reply.seq_num:
                        
                        if len(not_acked)==0:
                            step=window_size
                        elif abs(base - heapq.nsmallest(1, not_acked)[0]) > window_size - 1:
                            step = abs(seq_num_range % base - heapq.nsmallest(1, not_acked)[0])
                        elif abs(base - heapq.nsmallest(1, not_acked)[0]) <= window_size - 1:
                            step = abs(base - heapq.nsmallest(1, not_acked)[0])

                        index+=step

                        if index + window_size < len(packets):
                            
                            window = packets[index:index + window_size]
 
                        else:

                            window = packets[index:len(packets)]
                                    
                        

                        base= window[0].seq_num
                        print(window[0].act_num," ---- ",window[len(window)-1].act_num)
                        
                        tm.reset()
                
                        for item in window[-step:]:
                            senderSocket.sendto(pickle.dumps(item),(receiverIP, receiverPort))
                            not_acked.append(item.seq_num)



        

        return



send_parent()


In [ ]:
from google.colab import files
files.download(recv_file_name)
files.download('receiver_stats.txt')
files.download('sender_stats.txt')

# SEL. REPEAT


In [ ]:
from concurrent.futures import ThreadPoolExecutor, wait, as_completed

from time import sleep
from random import randint
from threading import Thread, Lock

lock=Lock()

def time_alarm(Packet, prob, time_out, senderSocket,pool,timers,dummy):
    time.sleep(time_out)
    
    return Packet, Packet.seq_num, senderSocket, prob, time_out, pool,timers,dummy
   
def resend(fn):
    
    packet,number,senderSocket, prob, time_out, pool, timers,dummy=fn.result()
    
    if decision(prob):
      with lock:
        try:
          senderSocket.sendto(pickle.dumps(packet),(receiverIP, receiverPort))
          print("resent",number,"actual",packet.act_num)
        except:
          pass
          
          
    else:
    
      timers[number]=pool.submit(time_alarm, packet, prob, time_out,senderSocket,pool,dummy+1)

            
            

In [ ]:
def is_in_range(num,start,step_size,range_):
  lim=[ i%range_ for i in range(start,start+step_size)]
  if num in lim:
    return True
  else:
    return False
  
def is_in_lrange(num,start,step_size,range_):
  lim=[ i%range_ for i in range(start-1,start-step_size)]
  if num in lim:
    return True
  else:
    return False
    

          

In [ ]:
#@title Parameters Tuning{ form-width: "27%" }
sent_file_name = 'Podington_Bear_-_Starling.mp3'  # @param {type:"string"}
recv_file_name = 'sel_rep.mp3'  # @param {type:"string"}
prob = 1  # @param {type:"slider", min:0, max:1, step:0.01}
window_size = 4  # @param {type:"integer"}
seq_num_range = 8  # @param {type:"integer"}
time_out = 0.1 # @param {type:"number"}

import heapq
import os
import time
import threading
from threading import Thread
start_time = time.time()

In [ ]:
def recv_child():
    stats_ = {}
    #UDP socket
    receiverSocket = socket(AF_INET, SOCK_DGRAM)
    #reuse address when done with it
    receiverSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    receiverSocket.bind(('', receiverPort))

    print ('--The receiver is ready to receive--\n')

    packets = []
    window = []
    received = []
    recv_base = 0
    flag_end = False
#   receiving while loop, message is a packet of the file sent
    while True:
        print(received)
        (message, senderAddress) = receiverSocket.recvfrom(8192)
        
        message = pickle.loads(message)
        #last packet
        if message.type_ == 0:
            flag_end = True
        #if packet received is within current window
        if is_in_lrange(message.seq_num, recv_base, window_size,
                        seq_num_range) or is_in_range(message.seq_num,
                recv_base, window_size, seq_num_range):
            print("YES in range:",message.act_num)
            if True:
                
                receiverSocket.sendto(pickle.dumps(ACK(message.seq_num,
                        0, message.act_num)), senderAddress)
                t6 = time.time()
                stats_['{0:.6f}'.format(t6 - start_time)] = \
                    'receiver sent ack for packet ' \
                    + str(message.seq_num) + ' actual: ' \
                    + str(message.act_num)
                print (stats_['{0:.6f}'.format(t6 - start_time)])
            #else if packet is dropped
            else:
                t6 = time.time()
                stats_['{0:.6f}'.format(t6 - start_time)] = \
                    'receiver dropped ack for packet ' \
                    + str(message.seq_num) + ' actual: ' \
                    + str(message.act_num)

        if message.seq_num in received:
            continue

        if message.seq_num<window_size:
          if (message.seq_num+seq_num_range- window_size) in received:
            received.remove(message.seq_num+seq_num_range- window_size)
        else:
          if (message.seq_num-window_size) in received:
            received.remove(message.seq_num-window_size)

        received.append(message.seq_num)

        if recv_base == message.seq_num:
            window.append(message)
            for (i, j) in zip(sorted(window)[0::1], sorted(window)[1::1]):
                if j.seq_num == (i.seq_num + 1) % seq_num_range \
                    or j.seq_num == i.seq_num + 1:
                    recv_base = j.seq_num
                    packets.append(i)
                    window.remove(i)

                else:
                    recv_base = (i.seq_num + 1) % seq_num_range
                    packets.append(i)
                    window.remove(i)

                    break

            if len(window) == 1 and recv_base == window[0].seq_num:
                recv_base = (window[0].seq_num + 1) % seq_num_range
                packets.append(window[0])
                window.remove(window[0])
                

            
        else:
            window.append(message)
            
        if packets:
          if flag_end:
            time.sleep(0.1)

            print ('--Whole file received successfully. File is ready to download--\n')

            get_stats('receiver_stats.txt', stats_)

            pkt_to_file(recv_file_name, packets)
            receiverSocket.close()
            break

    return



In [ ]:
#Sender
def send_parent():
    #sender process
    newpid = os.fork()
    #if in child
    if newpid == 0:
        recv_child()
    else:#parent
        stats_ = {}
        time.sleep(1)
        packets = file_to_pkt(file_name=sent_file_name, sw=False,
                              seq_num_range=seq_num_range)

        senderSocket = socket(AF_INET, SOCK_DGRAM)
        senderSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
        senderSocket.bind(('', senderPort))

        # ## initial setting of window
        print("length of packets:",len(packets))
        base = 0
        index = 0
        step = 0
        window = []
        timers = {}
        not_acked = []
        acked_num = 0

        if index + window_size < len(packets):
            window = packets[index:(index + window_size)]
        else:
            window = packets[index:len(packets)]

        for i in range(len(window)):
            #for each packet in window create an event object and  a timer/thread
            #key of these dics to get these object is packet sequence number
            senderSocket.sendto(pickle.dumps(window[i]),(receiverIP, receiverPort))
            timers[window[i].seq_num] = TimerReset(time_out,resend,[window[i],prob,start_time,stats_,senderSocket])
            timers[window[i].seq_num].start()
            not_acked.append(window[i].seq_num)

        while True:

            reply = None
            with lock:
              (reply, receiverAddress) = senderSocket.recvfrom(4096)
           
            reply = pickle.loads(reply)
            print("ack seq_num:",reply.seq_num,reply.real,"REC. ACK")
            if isinstance(reply,ACK):
               
                

                if (reply.seq_num in timers.keys()):

                    timers[reply.seq_num].cancel()
                    del timers[reply.seq_num]
                    try:
                      not_acked.remove(reply.seq_num)
                    except ValueError:
                      print(not_acked,reply.seq_num)

                    acked_num += 1

                    if acked_num == len(packets):
                        senderSocket.close()
                        break

                    if base == reply.seq_num:
                        if len(not_acked)==0:
                            step=window_size
                        elif abs(base - heapq.nsmallest(1, not_acked)[0]) > window_size - 1:
                            step = abs(seq_num_range % base - heapq.nsmallest(1, not_acked)[0])
                        elif abs(base - heapq.nsmallest(1, not_acked)[0]) <= window_size - 1:
                            step = abs(base - heapq.nsmallest(1, not_acked)[0])

                        index+=step

                        if index + window_size < len(packets):
                            
                            window = packets[index:index + window_size]
 
                        else:

                            window = packets[index:len(packets)]

                        

                        base= window[0].seq_num
                        print(window[0].act_num," ---- ",window[len(window)-1].act_num)

                        for item in window[-step:]:
                            senderSocket.sendto(pickle.dumps(item),(receiverIP, receiverPort))
                            
                            timers[item.seq_num] = TimerReset(time_out,resend,[item,prob,start_time,stats_,senderSocket])
                            timers[item.seq_num].start()
                            not_acked.append(item.seq_num)



        

        return




In [ ]:
send_parent()

In [ ]:
from google.colab import files
files.download(recv_file_name)